In [1]:
from PIL import Image
import os
import shutil
import numpy as np
# import cv2
import re
import pickle
import random
import time
# from tqdm import tqdm
import pandas as pd 
from glob import glob
from xml.etree.ElementTree import parse
# from pascal_voc_writer import Writer
import matplotlib.pyplot as plt

import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.auto import tqdm


#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler

#sklearn
from sklearn.model_selection import StratifiedKFold

#CV
import cv2

################# DETR FUCNTIONS FOR LOSS######################## 
import sys
sys.path.extend(['./detr/'])

from models.matcher import HungarianMatcher
from models.detr import SetCriterion
################################################################

#Albumenatations
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

#Glob
from glob import glob

import sys
sys.path.extend(['./util/'])
from data_parser import *
from image_prepro import *
# from torchvision import transforms

from visualize import *

In [2]:
# src_path = '/home/beomgon/Object_Detection/Dataset'
# # df = pd.DataFrame(columns=['label', 'org_label', 'bbox', 'xmin','ymin','xmax','ymax','width','height', 'file_name', 'path', 'h', 'w'])
# # df = pd.DataFrame(columns=['class','bbox', 'path'])
# data_list = []
# for dir_name in ['SS', 'SS2']:
#     dpath = os.path.join(src_path, dir_name)
# #     print(dpath)
#     dir_lists = os.listdir(dpath) 
# #     print(dir_lists)
    
#     for dir_list in dir_lists :
#         dpath2 = os.path.join(dpath, dir_list)
#         xml_list = [os.path.join(dpath2, d) for d in os.listdir(dpath2) if d.endswith(".xml")]
# #         print(xml_list)
#         for xml_path in xml_list :
#             parser = XMLParser(xml_path)
#             if (parser.height, parser.width) not in parser.rejection_size:  
#                 filename = parser.file_name
#                 img_path = '/'.join((xml_path.split('/')[:-1])) + '/' + filename
#                 image = cv2.imread(img_path)
#                 h, w, _ =  image.shape                
#                 for obj in parser.objects :
#     #                     obj_class = obj[0]
#     #                     if obj_class in rej_table :
#     #                         continue
#     #                     print(obj)
# #                     img_path = '/'.join((xml_path.split('/')[:-1])) + '/' + filename
#     #                 img_path = re.sub('xml', 'jpg', xml_path)
#     # #                     print(img_path)
#                     obj.append(parser.width)
#                     obj.append(parser.height)
#                     obj.append(parser.file_name)
#                     obj.append(img_path)
#                     obj.append(h)
#                     obj.append(w)
#                     data_list.append(obj)
# #                     h_list.append(h)
# #                     w_list.append(w)
#     # #                     print(obj)


In [3]:
# df = pd.DataFrame(data_list,columns=['label', 'org_label', 'bbox', 'xmin','ymin','xmax','ymax','width','height', 'file_name', 'path', 'h', 'w'])
# df = df[['label', 'org_label', 'width','height', 'h', 'w', 'bbox', 'xmin','ymin','xmax','ymax', 'file_name', 'path']]
# # print(df.head())
# # df = df.reset_index(drop=True)
# # df.drop(columns = ['index'], inplace=True)
# df.reset_index(drop=True, inplace=True)
# df.head()

In [4]:
# df.to_csv('df.csv', index=None)

In [5]:
df = pd.read_csv('df.csv')

In [6]:
df.drop(columns=['width', 'height'], inplace=True)
df.head()

,label,org_label,h,w,bbox,xmin,ymin,xmax,ymax,file_name,path
0,Carcinoma,Suamous cell carcinoma,4032,1960,"[2061, 1051, 2178, 1175]",2061,1051,2178,1175,20200623_100015.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...
1,HSIL,HSIL,4032,1960,"[1945, 870, 2024, 941]",1945,870,2024,941,20200420_100401.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...
2,HSIL,HSIL,4032,1960,"[2400, 947, 2484, 1020]",2400,947,2484,1020,20200420_100401.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...
3,HSIL,HSIL,4032,1960,"[2209, 845, 2307, 912]",2209,845,2307,912,20200420_100401.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...
4,HSIL,HSIL,4032,1960,"[2056, 811, 2144, 866]",2056,811,2144,866,20200420_100401.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...


In [7]:
# len(df[df['width'] != df['h']])

In [8]:
# (df[df['height'] != df['w']])

In [9]:
# 

In [10]:
switched_df = (df[(df['h'] == 1800) | (df['h'] == 1560)]).copy()
not_switched_df = (df[(df['h'] != 1800) & (df['h'] != 1560)]).copy()
print(len(switched_df))
print(len(not_switched_df))
print(len(df))
switched_df.reset_index(drop=True, inplace=True)
not_switched_df.reset_index(drop=True, inplace=True)

12765
92619
105384


In [11]:
import operator

width = []
height = []
xmax_list = list(not_switched_df.xmax)
xmin_list = list(not_switched_df.xmin.apply(lambda x : -1*x))
ymax_list = list(not_switched_df.ymax)
ymin_list = list(not_switched_df.ymin.apply(lambda x : -1*x))

height = list(map(operator.add, xmax_list, xmin_list))
width = list(map(operator.add, ymax_list, ymin_list))
not_switched_df['box_width'] = width
not_switched_df['box_height'] = height
not_switched_df['x_min'] = list(not_switched_df.ymin)
not_switched_df['y_min'] = list(not_switched_df.xmin)
not_switched_df.drop(columns=['bbox', 'xmax', 'ymax', 'xmin', 'ymin'], inplace=True)
not_switched_df.head()
not_switched_df.columns

Index(['label', 'org_label', 'h', 'w', 'file_name', 'path', 'box_width',
       'box_height', 'x_min', 'y_min'],
      dtype='object')

In [12]:
# not_crop_df[not_crop_df['height'] == 1560]

In [13]:
import operator

width = []
height = []
xmax_list = list(switched_df.xmax)
xmin_list = list(switched_df.xmin.apply(lambda x : -1*x))
ymax_list = list(switched_df.ymax)
ymin_list = list(switched_df.ymin.apply(lambda x : -1*x))

height = list(map(operator.add, ymax_list, ymin_list))
width = list(map(operator.add, xmax_list, xmin_list))
switched_df['box_width'] = width
switched_df['box_height'] = height
switched_df['x_min'] = list(switched_df.xmin)
switched_df['y_min'] = list(switched_df.ymin)
switched_df.drop(columns=['bbox', 'xmax', 'ymax', 'xmin', 'ymin'], inplace=True)
switched_df.head()
switched_df.columns

Index(['label', 'org_label', 'h', 'w', 'file_name', 'path', 'box_width',
       'box_height', 'x_min', 'y_min'],
      dtype='object')

In [14]:
df1 = pd.concat([switched_df,not_switched_df])
df1.reset_index(drop=True, inplace=True)
# print(df.tail())
df1.columns

Index(['label', 'org_label', 'h', 'w', 'file_name', 'path', 'box_width',
       'box_height', 'x_min', 'y_min'],
      dtype='object')

In [15]:
# df1[df1['box_width'] == 0]

In [16]:
len(df1)

105384

In [17]:
# drop NA cell
# print((len(df1)))
df1.label = df1.label.apply(lambda x : np.nan if x == '' else x)
df1.dropna(subset=['label'], inplace=True)
df1.reset_index(drop=True, inplace=True)
print((len(df1)))

100894


In [18]:
print(df1.shape)
df1 = df1[df1['box_height'] != 0]
print(df1.shape)
df1.reset_index(drop=True, inplace=True)
df1[df1['box_width'] == 0]

(100894, 10)
(100893, 10)


,label,org_label,h,w,file_name,path,box_width,box_height,x_min,y_min


In [19]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        
    @property
    def avg(self):
        return (self.sum / self.count) if self.count>0 else 0

In [20]:
# df_data = df.groupby('file_name')
# images = list(set(df.file_name))
df_data = df1.groupby('path')
images = list(set(df1.path))
len(images)
# len(df_data)
# len(df_data.groups)
# df_data.get_group('20200420_100401.jpg')

18720

In [21]:
def label_to_id(x) :
    x = str(x)
    if x == 'ASCUS' :
        return 0
    elif x == 'Carcinoma' :
        return 1
    elif x == 'HSIL' :
        return 2
    elif x == 'LSIL' :
        return 3
    elif x == 'Normal' :
        return 4
    
df1['label_id'] = df1.label.apply(lambda x : label_to_id(x))
df1.head()

,label,org_label,h,w,file_name,path,box_width,box_height,x_min,y_min,label_id
0,HSIL,HSIL,1560,1632,20200513_104735.png,/home/beomgon/Object_Detection/Dataset/SS/04/2...,114,105,1040,715,2
1,HSIL,HSIL,1560,1632,20200513_104735.png,/home/beomgon/Object_Detection/Dataset/SS/04/2...,79,78,1061,845,2
2,HSIL,HSIL,1560,1632,20200513_104735.png,/home/beomgon/Object_Detection/Dataset/SS/04/2...,75,90,509,697,2
3,HSIL,HSIL,1560,1632,20200513_145534.png,/home/beomgon/Object_Detection/Dataset/SS/04/2...,146,200,865,798,2
4,HSIL,HSIL,1560,1632,20200518_104822.png,/home/beomgon/Object_Detection/Dataset/SS/04/2...,118,127,515,494,2


In [22]:
def get_data(img_id):
    if img_id not in df_data.groups:
        return dict(image_id=img_id, source='', boxes=list())
    
    data  = df_data.get_group(img_id)
    labels = data.label.values
#     source = data.source.values
#     source = np.unique(data.source.values)
#     assert len(source)==1, 'corrupted data: %s image_id has many sources: %s' %(img_id,source)
#     source=source[0]
    boxes = data[['x_min','y_min','box_width','box_height']].values
#     boxes = list(data.bbox)
    label_id = data.label_id.values
    return dict(image_id = img_id, labels=labels, label_id=label_id, boxes = boxes)

image_list = [get_data(img_id) for img_id in images]

In [23]:
# image_list.remove('/home/beomgon/Object_Detection/Dataset/SS/04/20200513_101513.png')
# del image_list[185]
# del image_list[4144]


In [24]:
transforms = A.Compose([
#     A.RandomCrop(width=450, height=450),
#     A.HorizontalFlip(p=1),
    A.CenterCrop(1560,1560, True,1),
#     A.RandomCrop(height=512, width=512, p=1),
    A.RandomCrop(height=384, width=384, p=1),
#     A.Resize(height=512, width=512, p=1),
#     A.Resize(height=256, width=256, p=1),
    
    A.OneOf([
    A.HorizontalFlip(p=0.8),
    A.VerticalFlip(p=0.8),
    A.RandomRotate90(p=0.8),        
    ], p=1),
# #     A.GaussNoise(p=0.5),
#     A.OneOf([
#     A.OpticalDistortion(p=0.7),
#     A.GaussNoise(p=0.7)        
#     ], p=1),
#     A.RandomBrightnessContrast(p=0.2),
#     A.Normalize(max_pixel_value=1),
#     ToTensorV2(p=1.0),
    A.pytorch.ToTensor(),
# ], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.5))
], p=1.0, bbox_params=A.BboxParams(format='coco', min_area=0, min_visibility=0.5, label_fields=['labels'])) 

In [25]:
# DIR_TRAIN = '../input/global-wheat-detection/train'
class WheatDataset(Dataset):
    def __init__(self,image_list,transforms=None):
        self.images = image_list
        self.transforms = transforms
        self.img_ids = {x['image_id']:i for i,x in enumerate(image_list)}
        
    def get_indices(self,img_ids):
        return [self.img_ids[x] for x in img_ids]
        
    def __len__(self) -> int:
        return len(self.images)
    
    def __getitem__(self,index):
        record = self.images[index]
        image_id = record['image_id']
#         print(image_id)

        image = cv2.imread(image_id, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = cv2.flip(image, 1)
        image /= 255.0
        
        # DETR takes in data in coco format 
        boxes = record['boxes'] 
#         print('boxes', boxes)
        
#         labels =  np.zeros(len(boxes), dtype=np.int32)
        labels = record['label_id']
#         labels = np.array(labels, dtype=np.int32)
#         print('label', type(labels))
#         print(labels)
#         print(image.shape)

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }
            sample = self.transforms(**sample)
            image  = sample['image']
            boxes  = sample['bboxes']
            labels = sample['labels']

        _,h,w = image.shape
#         print(image.shape)
#         h = w = 256

        boxes = A.augmentations.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
#         print('boxes', boxes)
        ## detr uses center_x,center_y,width,height !!
        if len(boxes)>0:
            boxes = np.array(boxes)
            boxes[:,2:] /= 2
            boxes[:,:2] += boxes[:,2:]
        else:
            boxes = np.zeros((0,4))
    
        target = {}
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels, dtype=torch.int64)
        target['image_id'] = torch.tensor([index])
        
        return image, target, image_id

In [26]:
train_ds = WheatDataset(image_list,transforms)
# valid_ds = WheatDataset(image_list,get_valid_transforms())

In [27]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [28]:
batch_size = 32
train_data_loader = DataLoader(
#     torch.utils.data.Subset(train_ds,train_indexes),
    train_ds,
    batch_size=batch_size,
#     shuffle=True,
    shuffle=False,
    num_workers=1,
    collate_fn=collate_fn
#     collate_fn=None
)

In [29]:
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes+1)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

In [30]:
n_folds = 5
seed = 42
null_class_coef = 0.5
# num_classes = 1
num_queries = 100
# BATCH_SIZE = 2
LR = 5e-5
lr_dict = {'backbone':0.1,'transformer':1,'embed':1,'final': 5}
epochs = 10
max_norm = 0
model_name = 'detr_resnet50'

In [31]:
'''
code taken from github repo detr , 'code present in engine.py'
'''

# matcher = HungarianMatcher(cost_giou=2,cost_class=1,cost_bbox=5)
matcher = HungarianMatcher()

weight_dict = {'loss_ce': 1, 'loss_bbox': 5 , 'loss_giou': 2}

losses = ['labels', 'boxes', 'cardinality']

In [32]:
num_classes = 5
criterion = SetCriterion(num_classes, 
                         matcher, weight_dict, 
                         eos_coef = null_class_coef, 
                         losses=losses)

model = DETRModel(num_classes=5, num_queries=num_queries)
# model.summary()
learning_rate = 0.00005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Using cache found in /home/beomgon/.cache/torch/hub/facebookresearch_detr_master


In [33]:
scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / ((epoch/4) + 1)
)


In [34]:
device = torch.device('cuda')
criterion = criterion.to(device)
model = DETRModel(num_classes=5, num_queries=num_queries)
model = model.to(device)
model.train()
criterion.train()
tk0 = tqdm(train_data_loader, total=len(train_data_loader),leave=False)
log = None
device='cuda'
im = []
path_list = []
for epoch in range(epochs):
    for step, (images, targets, image_ids) in enumerate(tk0):

        im.append(step)
        path_list.append(image_ids)
        images = list(image.to(device) for image in images)
    #     print(images[0])
    #     print(type(images))
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        output = model(images)

        loss_dict = criterion(output, targets)
        if log is None:
            log = {k:AverageMeter() for k in loss_dict}
            log['total_loss'] = AverageMeter()
            log['avg_prec'] = AverageMeter()

        weight_dict = criterion.weight_dict

        total_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        optimizer.zero_grad()

        total_loss.backward()

    #     if max_norm > 0:
    #         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        optimizer.step()    
        log['total_loss'].update(total_loss.item(),batch_size)
        for k,v in loss_dict.items():
            log[k].update(v.item(),batch_size)

    #     log['avg_prec'].update(challenge_metric(output,targets),batch_size)

        tk0.set_postfix({k:v.avg for k,v in log.items()})         
    scheduler.step()


Using cache found in /home/beomgon/.cache/torch/hub/facebookresearch_detr_master


KeyboardInterrupt: 

In [ ]:
log['total_loss'].avg

In [ ]:
# print(969/1960)
# print(1577/1560)
# print(1558/1560)

In [ ]:
im[-1]

In [ ]:
im[-1]

In [ ]:
# path_list[-4:]

In [ ]:
# image_list[0]

In [ ]:
# image_list[im[-3]:im[-3]+4]

In [ ]:
print((1529+135)/1560)
print((1529+135)/1632)
print(1582/1560)
print(1614/1560)
# print(1562/1560)

In [ ]:
im[-1]

In [ ]:
# path_list[2037:2041]

In [ ]:
# image_list[2037:2041]

In [ ]:
# df1[df1['path']== '/home/beomgon/Object_Detection/Dataset/SS2/03(200908-10-normal)/20200908_104214.jpg']